In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH17-LangGraph-Use-Cases")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH17-LangGraph-Use-Cases


### TO DO

#### 음성 인식 주석과 문서의 관련성을 검사하고 자체 피드백후 더 정확한 주석 반영하기

#### 1. gpt api를 이용해서 구현하기

#### 2. solar-pro api를 이용해서 구현하기

#### 3. 비동기식으로 코드를 변경하여서, 반응성을 높이기 (추후과정)

In [10]:
import myUpstageRAG
import myPDFparser


# 내 rag_pipeline 생성.
DOCUMENT_PATH = "data/10 Vector Calculus.pdf"
document = myPDFparser.upstageParser2Document(file_path=DOCUMENT_PATH)

Gradients
of
Matrices
[Document(metadata={'page': 1}, page_content='■\nUseful\nidentities:\nGradients\nof\nMatrices\n이 도표는 행렬과 관련된 다양한 수학적 연산과 개념을 시각적으로 표현하고 있습니다. 도표는 10개의 주요 항목으로 구성되어 있으며, 각각의 항목은 수학적 표현과 함께 설명이 포함되어 있습니다. 가장 왼쪽 항목은 대각선 행렬과 그 역행렬을 나타내며, 그 옆에는 행렬의 행렬식과 트레이스가 설명되어 있습니다. 다른 항목들로는 행렬의 역행렬과 관련된 행렬 연산, 행렬의 고유값과 고유벡터, 그리고 행렬의 전치 및 트랜스포즈에 대한 정보가 포함되어 있습니다. 도표는 수학적 표현과 함께 설명이 포함되어 있어서, 행렬과 관련된 다양한 개념을 이해하는 데 도움이 됩니다. 전체적으로는 대칭 행렬에 대한 연산과 특성을 설명하고 있으며, 이를 통해 행렬의 다양한 특성과 연산을 이해할 수 있습니다.\n12'), Document(metadata={'page': 2}, page_content='■\nThen\ncompute\nazTWz\ndz\nas\ndz\n=\n2(x-As)\nTW\n·\n(-A)\n■\n■\nas\na\nLet\ndz\nZ\nazTWz\n=x-As,\nAs)\n(x-\n= ZT(W + WT)\nand\nuse\n=\n「W(x - As)\naxT\ndx\nBx\n= x\n2(x-As)\n\nTW\n)\n:\n(B+B\n= -2(x - As) WA\na\nfor\nax\nf(X)-1= -f(X)\nsymmetric\n(5.108)\nW\nax\naf(X)\nf(X)-1\n■\nClarification\nof\nsome\nidentities:\nGradients\nof\nMatrices\n(5.102)\n0\n=\nax\nai\n=\nax\n0\n(f(X)-1f(X))\n=\n(。\nf(X)-1)\nf(X)\n+\nf(X)\n-1\n(�f(X))\n13')]


## 답변 생성 chain 생성

In [4]:
from myUpstageRAG import myRAG
rag_pipeline = myRAG(document)


In [5]:
from langchain_teddynote.models import get_model_name, LLMs
from langchain_openai import ChatOpenAI

# 최신 모델 가져오기
GPT4o = get_model_name(LLMs.GPT4o)

# 모델 초기화
llm = ChatOpenAI(model=GPT4o)

## 문서와 질의의 관련성 평가
llm (gtp-4) 모델로함

In [6]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_teddynote.models import get_model_name, LLMs
from pydantic import BaseModel, Field
# 검색된 문서의 관련성 평가.
# 검색된 문서가 질문과 관련이 있는지 여부 평가!


# 모델 이름 가져오기
MODEL_NAME = get_model_name(LLMs.GPT4)


# 검색된 문서의 관련성 여부를 이진 점수로 평가하는 데이터 모델
class GradeDocuments(BaseModel):
    """A binary score to determine the relevance of the retrieved document."""

    # 문서가 질문과 관련이 있는지 여부를 'yes' 또는 'no'로 나타내는 필드
    binary_score: str = Field(
        description="Documents are relevant to the question, 'yes' or 'no'"
    )


# LLM 초기화
llm = ChatOpenAI(model=MODEL_NAME, temperature=0)

# GradeDocuments 데이터 모델을 사용하여 구조화된 출력을 생성하는 LLM
structured_llm_grader = llm.with_structured_output(GradeDocuments)

# 시스템 프롬프트 정의
system = """You are a grader assessing relevance of a retrieved document to a user question. \n 
    If the document contains keyword(s) or semantic meaning related to the question, grade it as relevant. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."""

# 채팅 프롬프트 템플릿 생성
grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
    ]
)

# Retrieval 평가기 초기화
retrieval_grader = grade_prompt | structured_llm_grader

In [7]:
# 질문 정의
question = "삼성전자가 개발한 생성AI 에 대해 설명하세요."

# 문서 검색
docs = rag_pipeline.retriever.invoke(question)


In [8]:
docs

[Document(id='afd53868-b4b9-4f12-ba8c-1dceb40e38a0', metadata={'page': 1}, page_content='■\nUseful\nidentities:\nGradients\nof\nMatrices\n이 도표는 행렬 및 벡터 연산에 대한 여러 수학적 정체성과 공식을 보여준다. 왼쪽에는 행렬과 벡터의 다양한 조합과 연산이 표시되어 있으며, 오른쪽에는 각 연산의 결과가 나와있다. 구체적으로, 도표에는 행렬식, 전치, 역행렬, 트레이스, 대칭 행렬 등 다양한 개념과 연산이 포함되어 있다. 각 연산은 수식과 함께 도식화된 방법으로 표현되어 시각적으로 이해를 돕는다. 전체적으로, 이 도표는 선형대수학에서 사용되는 다양한 행렬 및 벡터 연산과 그 결과를 종합적으로 보여준다.\n12'),
 Document(id='b0657754-315d-40d7-948b-3d857ccb4c03', metadata={'page': 2}, page_content='■\nThen\ncompute\nazTWz\ndz\nas\ndz\n=\n2(x-As)\nTW\n·\n(-A)\n■\n■\nas\na\nLet\ndz\nZ\nazTWz\n=x-As,\nAs)\n(x-\n= ZT(W + WT)\nand\nuse\n=\n「W(x - As)\naxT\ndx\nBx\n= x\n2(x-As)\n\nTW\n)\n:\n(B+B\n= -2(x - As) WA\na\nfor\nax\nf(X)-1= -f(X)\nsymmetric\n(5.108)\nW\nax\naf(X)\nf(X)-1\n■\nClarification\nof\nsome\nidentities:\nGradients\nof\nMatrices\n(5.102)\n0\n=\nax\nai\n=\nax\n0\n(f(X)-1f(X))\n=\n(。\nf(X)-1)\nf(X)\n+\nf(X)\n-1\n(�f(X))\n13')]

In [9]:

# 검색된 문서 중 1번 index 문서의 페이지 내용을 추출
doc_txt = docs[1].page_content

# 검색된 문서와 질문을 사용하여 관련성 평가를 실행하고 결과 출력
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))

binary_score='no'


## 쿼리 재작성
웹검색을 최적화 하기 위한 질문 재작성 단계

In [18]:
import requests
from openai import OpenAI # openai==1.52.2
api_key = "up_qAeV4wltdGnqdmxavmqmxXyRBpmcZ" # ex: up_xxxYYYzzzAAAbbbCCC

## llm 생성.
client = OpenAI(
    api_key=api_key,
    base_url="https://api.upstage.ai/v1"
)

def chat_with_solar(messages):
    response = client.chat.completions.create(
        model="solar-pro",
        messages=messages
    )
    return response.choices[0].message.content

# Query Rewrite 시스템 프롬프트
system = """You a question re-writer that converts an input question to a better version that is optimized 
for web search. Look at the input and try to reason about the underlying semantic intent / meaning.

"Here is the initial question: \n\n {question} \n Formulate an improved question."
"""
new_question = system.format(question=question)
new_messages = [{"role": "user", "content": new_question}]
# response = chat_with_solar(new_messages)
# print("Assistant:", response)

Assistant: "Provide an overview of the generative AI developed by Samsung Electronics."


In [19]:
# 실행 및 결과 확인
print(f'[원본 질문]: "{question}"')
print("[쿼리 재작성]:", chat_with_solar(new_messages))

[원본 질문]: "삼성전자가 개발한 생성AI 에 대해 설명하세요."
[쿼리 재작성]: "What is the description of the generative AI developed by Samsung Electronics?"


In [20]:
# 웹 검색 도구 초기화
from langchain_teddynote.tools.tavily import TavilySearch

# 최대 검색 결과를 3으로 설정
web_search_tool = TavilySearch(max_results=3)

In [21]:
# 웹 검색 도구 실행
results = web_search_tool.invoke({"query": new_question})
print(results)

[{'title': '"Ai 홈으로 일상을 혁신하다" 삼성전자, 2025년형 비스포크 Ai 라인업 공개', 'url': 'https://news.samsung.com/kr/ai-홈으로-일상을-혁신하다-삼성전자-2025년형-비스', 'content': '삼성전자가 28일 \'웰컴 투 비스포크 AI(Welcome To Bespoke AI)\' 행사를 개최하고 일상을 혁신하는 새로운 AI 가전 제품과 서비스를 대거 공개했다. 삼성전자 DA사업부 개발팀장 문종승 부사장은 "삼성전자는 하드웨어 혁신을 넘어 기기 간 연결과 AI 기술을 통해,', 'score': 0.7355091, 'raw_content': None}, {'title': '2025년, 생성형 Ai 트렌드 전망 | 인사이트리포트 | 삼성sds', 'url': 'https://www.samsungsds.com/kr/insights/outlook-for-generative-ai-in-2025.html', 'content': '클라우드 보안 서비스 클라우드 보안 서비스 클라우드 보안 관제 서비스 클라우드 보안 진단 서비스 클라우드 보안 솔루션 운영 서비스 Security 다양한 클라우드 환경에서 데이터와 서비스를 안전하게 보호하는 자동화 된 보안 서비스 AI/ML 쉽고 편리하게 ML/DL 모델 개발 및 학습 환경을 구축할 수 있는 AI 서비스 삼성SDS 글로벌 물류 서비스 삼성SDS, 더 정교해진 AI 분석·예측으로 ‘데이터 기반 디지털 물류’ 이끈다.자세히 보기 클라우드 보안 서비스 클라우드 보안 서비스 클라우드 보안 관제 서비스 클라우드 보안 진단 서비스 클라우드 보안 솔루션 운영 서비스 AI 준비 상태로 만드는 작업을 빨리 시작할수록 2025년 출시될 새로운 RAG 서비스를 더 빨리 활용할 수 있습니다. 2025년에는 또한 마이크로소프트 애저, AWS, 구글 클라우드 등 클라우드 기업의 AI 제품이 발전하면서 경쟁력 있는 가격이 형성될 수 있습니다.', 'score': 0.73322314, 'raw_con

In [24]:
results[1]["content"]

'클라우드 보안 서비스 클라우드 보안 서비스 클라우드 보안 관제 서비스 클라우드 보안 진단 서비스 클라우드 보안 솔루션 운영 서비스 Security 다양한 클라우드 환경에서 데이터와 서비스를 안전하게 보호하는 자동화 된 보안 서비스 AI/ML 쉽고 편리하게 ML/DL 모델 개발 및 학습 환경을 구축할 수 있는 AI 서비스 삼성SDS 글로벌 물류 서비스 삼성SDS, 더 정교해진 AI 분석·예측으로 ‘데이터 기반 디지털 물류’ 이끈다.자세히 보기 클라우드 보안 서비스 클라우드 보안 서비스 클라우드 보안 관제 서비스 클라우드 보안 진단 서비스 클라우드 보안 솔루션 운영 서비스 AI 준비 상태로 만드는 작업을 빨리 시작할수록 2025년 출시될 새로운 RAG 서비스를 더 빨리 활용할 수 있습니다. 2025년에는 또한 마이크로소프트 애저, AWS, 구글 클라우드 등 클라우드 기업의 AI 제품이 발전하면서 경쟁력 있는 가격이 형성될 수 있습니다.'

In [ ]:
from typing import Annotated, List
from typing_extensions import TypedDict


# 상태 정의
class GraphState(TypedDict):
    question: Annotated[str, "The question to answer"]
    generation: Annotated[str, "The generation from the LLM"]
    web_search: Annotated[str, "Whether to add search"]
    documents: Annotated[List[str], "The documents retrieved"]  # 검색된 documents

In [ ]:
from langchain.schema import Document

# 내가 정의한 RAG chain을 사용하기 위한 메세지 형태.
# 형태 : messages = [{"role": "user", "content": formatted_prompt}]
def formatting_question(question : str):
    return  [{"role": "user", "content": question}]


# 문서 검색 노드
# 문서 내에서 
def retrieve(state: GraphState):
    print("\n==== RETRIEVE ====\n")
    question = state["question"]

    # 문서 검색 수행
    documents = rag_pipeline.retriever.invoke(question)
    return {"documents": documents}


# 답변 생성 노드
def generate(state: GraphState):
    print("\n==== GENERATE ====\n")
    question = state["question"]

    # RAG를 사용한 답변 생성
    generation = rag_pipeline.RAG_chain_invoke(question)
    return {"generation": generation}


# 문서 평가 노드
def grade_documents(state: GraphState):
    print("\n==== [CHECK DOCUMENT RELEVANCE TO QUESTION] ====\n")
    question = state["question"]
    documents = state["documents"]

    # 필터링된 문서
    filtered_docs = []
    relevant_doc_count = 0

    for d in documents:
        # Question-Document 의 관련성 평가
        score = retrieval_grader.invoke(
            {"question": question, "document": d.page_content}
        )
        grade = score.binary_score

        if grade == "yes":
            print("==== [GRADE: DOCUMENT RELEVANT] ====")
            # 관련 있는 문서를 filtered_docs 에 추가
            filtered_docs.append(d)
            relevant_doc_count += 1
        else:
            print("==== [GRADE: DOCUMENT NOT RELEVANT] ====")
            continue

    # 관련 문서가 없으면 웹 검색 수행
    web_search = "Yes" if relevant_doc_count == 0 else "No"
    return {"documents": filtered_docs, "web_search": web_search}


# 쿼리 재작성 노드
# myRAG에 맞게 재작성 필요할듯?
def query_rewrite(state: GraphState):
    print("\n==== [REWRITE QUERY] ====\n")
    question = state["question"]

    # 질문 재작성
    better_question = myRAG.invoke(question)
    return {"question": better_question}


# 웹 검색 노드
def web_search(state: GraphState):
    print("\n==== [WEB SEARCH] ====\n")
    question = state["question"]
    documents = state["documents"]

    # 웹 검색 수행
    docs = web_search_tool.invoke({"query": question})
    # 검색 결과를 문서 형식으로 변환
    web_results = "\n".join([d["content"] for d in docs])
    web_results = Document(page_content=web_results)
    documents.append(web_results)

    return {"documents": documents}